In [6]:
import pygame,random,threading
from tkinter import Scale, Tk, messagebox,HORIZONTAL
import tkinter as tk

class MazeGenerator:
    def __init__(self):
        pygame.init()
        self.clock = pygame.time.Clock()
        self.tile_size=23
        self.margin=1

        match goto:=2:
            case 1:#option one: design rows and cols yourself
                self.maze_rows,self.maze_cols=13,13# both rows and cols better be odd nums
                w,h= self.maze_cols*self.tile_size,self.maze_rows*self.tile_size
                self.window = pygame.display.set_mode((w,h),depth=999)
                self.background= pygame.Surface(self.window.get_size())
                self.auto_maze_padding_y,self.auto_maze_padding_x=(h-self.maze_rows*self.tile_size)/2,(w-self.maze_cols*self.tile_size)/2
            case 2:#option two: design width and height yourself
                self.window = pygame.display.set_mode((827, 871),depth=999)
                self.background= pygame.Surface(self.window.get_size())
                (w,h)= self.window.get_size()
                self.maze_rows,self.maze_cols=h // self.tile_size,w // self.tile_size
                self.auto_maze_padding_y,self.auto_maze_padding_x=(h-self.maze_rows*self.tile_size)/2,(w-self.maze_cols*self.tile_size)/2

        pygame.display.set_caption(f"width:{w},height:{h},rows:{self.maze_rows},cols:{self.maze_cols}")
        # pygame.display.set_allowed(FULLSCREEN | DOUBLEBUF | HWSURFACE | NOFRAME | TOPMOST)
        # No use anymore

        # 创建棋盘格状的背景
        # tiles = [((x * tile_size, y * tile_size, tile_size, tile_size), bgcolor1 if (x + y) % 2 == 0 else bgcolor2) for x in range((w + tile_size - 1) // tile_size) for y in range((h + tile_size - 1) // tile_size)]
        self.tiles=[]

        # Try to block the edges
        # mask=[[1 if x==0 or x==maze_rows-1 or y==0 or y==maze_cols-1 else 0  for y in range(maze_cols)] for x in range(maze_rows)]
        self.mask=[[1 for y in range(self.maze_cols)] for x in range(self.maze_rows)]

        self.toggle=1
        self.toggle_intensity=0
        self.ping=15
        self.generate_start=0
        self.path=[(1,1)]
        self.mask[1][1]=0

        
        self.quantum_tunneling=0.98
        
        print("MazeGenerator Class instanced")


    def mask2tiles(self):
        if self.generate_start!=-1:
            self.generate_start+=1
            if self.generate_start%self.ping==0:
                self.generate_start=0
                self.generate_path()
        self.tiles=[]
        toggle_node=self.path[-1]
        for x in range(self.maze_rows):
            for y in range(self.maze_cols):
                if (x,y)==toggle_node:
                    if self.toggle_intensity+self.toggle/self.ping>1:
                        self.toggle=-1
                    elif self.toggle_intensity+self.toggle/self.ping<0:
                        self.toggle=1
                    self.toggle_intensity=self.toggle_intensity+self.toggle/self.ping
                    fill_color=(0,255*self.toggle_intensity,0)
                else:
                    match self.mask[x][y]:
                        case 0: fill_color=(0, 0, 0)
                        case 1: fill_color=(192, 192, 192)
                self.tiles.append((( y * self.tile_size+self.margin+self.auto_maze_padding_y,x * self.tile_size+self.margin+self.auto_maze_padding_x, self.tile_size-self.margin, self.tile_size-self.margin),fill_color))
        # 为棋盘添加小格子
        for rect, color in self.tiles:
            pygame.draw.rect(self.background, color, rect)
        self.window.blit(self.background, (0, 0))

    def return_xy(self,pos):
        '''Returns the index of row and col'''
        return (pos[1]-self.auto_maze_padding_y)//self.tile_size,(pos[0]-self.auto_maze_padding_x)//self.tile_size

    def generate_path(self):
        def call_back():
            available_direction=[]
            node_row,node_col=self.path[-1]
            if node_row-2>0:
                if self.mask[node_row-2][node_col]!=0  or random.random()>self.quantum_tunneling:
                    available_direction.append(0)
            if node_col-2>0:
                if self.mask[node_row][node_col-2]!=0 or random.random()>self.quantum_tunneling:
                    available_direction.append(1)
            if node_row+3<self.maze_rows:
                if self.mask[node_row+2][node_col]!=0 or random.random()>self.quantum_tunneling:
                    available_direction.append(2)
            if node_col+3<self.maze_cols:
                if self.mask[node_row][node_col+2]!=0 or random.random()>self.quantum_tunneling:
                    available_direction.append(3)

            if len(available_direction)==0:
                return

            direction=random.sample(available_direction,k=1)[0]
            match direction:
                case 0:next_coord=((node_row-2,node_col),(node_row-1,node_col))
                case 1:next_coord=((node_row,node_col-2),(node_row,node_col-1))
                case 2:next_coord=((node_row+2,node_col),(node_row+1,node_col))
                case 3:next_coord=((node_row,node_col+2),(node_row,node_col+1))
            self.path.append(next_coord[0])
            for every_coord in next_coord:
                self.mask[every_coord[0]][every_coord[1]]=0
            call_back()
        call_back()
        if len(self.path)>1: 
            self.path.pop() 
        else: 
            self.generate_start=-1
            self.show_message("Call-back reaches end, Maze has been generated !")

    def start(self):
        run = True
        while run:
            self.clock.tick(60)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    run = False
                elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                        ## if mouse is pressed get position of cursor ##
                        pos = pygame.mouse.get_pos()
                        ## check if cursor is on tiles ##
                        pygame.display.set_caption(f'Clicked, Mask coord as {self.return_xy(pos)}')

            self.mask2tiles()
            # 将棋盘背景绘制到窗口上
            pygame.display.flip()
        pygame.quit()

    def exit(self):
        pygame.quit()

    def __del__(self):
        print("MazeGenerator Class instance destroyed")


class Messager(Tk):
    def __init__(self):
        super().__init__()
        self.title("Messager Window")
        self.withdraw()
        print("Messager Class instanced")
    
    def __del__(self):
        print("Messager Class instance destroyed")

    def show_message(self, message):
        messagebox.showinfo("Message", message)

# class SlideBar(Tk):
#     def __init__(self) -> None:
#         super().__init__()
#         self.title('Slide Bar For Ping or fps')
#         self.geometry("300x100")
#         # w = Scale(self, from_=0, to=42)
#         # w.pack()
#         w = Scale(self, from_=0, to=200, orient=HORIZONTAL)
#         w.pack()
#         print("SlideBar2 Class instanced")

    
#     def __del__(self):
#         print("SlideBar Class instance destroyed")

class MazeGenerator_Ver_1(MazeGenerator,Messager):
    def __init__(self):
        MazeGenerator.__init__(self)
        Messager.__init__(self)
        # SlideBar.__init__(self)
    
    def __del__(self):
        print("Holder Class instance destroyed")



maze_generator=MazeGenerator_Ver_1()
maze_generator.start()


MazeGenerator Class instanced
Messager Class instanced
